In [38]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Charger le jeu de données
df = pd.read_csv('data/df_merged_2020-2022.csv', sep=',', index_col="Num_Acc", low_memory=False)

df = df.replace('\xa0', '', regex=True)

# Nettoyer les données
# Remplacer les valeurs non numériques par NaN et les supprimer
df = df.apply(pd.to_numeric, errors='coerce')

In [39]:
df.head(5)

,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,senc,catv,...,Casque,Dispositif enfants,Gilet réfléchissant,Airbag,Gants,Autre,date,weekday,heure,age
Num_Acc,,,,,,,,,,,,,,,,,,,,,
202200000001,813952,NaN,1,1,3.0,1.0,2008.0,5.0,1.0,2.0,...,True,False,False,False,False,False,NaN,NaN,16,16.0
202200000001,813953,NaN,1,1,1.0,1.0,1948.0,5.0,1.0,7.0,...,False,False,False,False,False,False,NaN,NaN,16,76.0
202200000002,813950,NaN,1,1,4.0,1.0,1988.0,9.0,2.0,7.0,...,False,False,False,False,False,False,NaN,NaN,8,36.0
202200000002,813951,NaN,1,1,1.0,1.0,1970.0,4.0,2.0,10.0,...,False,False,False,False,False,False,NaN,NaN,8,54.0
202200000003,813948,NaN,1,1,1.0,1.0,2002.0,0.0,2.0,7.0,...,False,False,False,False,False,False,NaN,NaN,17,22.0


In [40]:
# Séparer les caractéristiques (features) et la cible (target)
X = df.drop('grav', axis=1)  # Remplacez 'target_column' par le nom de votre colonne cible
y = df['grav']  # Remplacez 'target_column' par le nom de votre colonne cible

# Diviser le jeu de données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser le modèle RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [43]:
# Définir la grille des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

In [44]:
# Entraîner GridSearchCV
grid_search.fit(X_train, y_train)

# Meilleurs hyperparamètres
print(f"Best parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 48 candidates, totalling 144 fits


Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq\\backend\\cython\\checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 

KeyboardInterrupt



In [35]:
# Utiliser le meilleur modèle pour prédire les valeurs sur l'ensemble de test
best_rf_model = grid_search.best_estimator_
y_pred = best_rf_model.predict(X_test)

In [37]:
# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.6747540983606557
Classification Report:
              precision    recall  f1-score   support

         1.0       0.72      0.84      0.78     29052
         2.0       0.41      0.04      0.07      1659
         3.0       0.53      0.42      0.47      9532
         4.0       0.65      0.62      0.64     25332

    accuracy                           0.67     65575
   macro avg       0.58      0.48      0.49     65575
weighted avg       0.66      0.67      0.66     65575

